In [8]:
# !pip install psycopg2-binary
import psycopg2
from psycopg2.errors import UniqueViolation 
from xml import sax

In [9]:
# connect to database
connection = psycopg2.connect("dbname = dblp user = dblpuser password = dblp_password")

# cursor to perform operations in postgres
cursor = connection.cursor()

# reset
cursor.execute("""
DROP TABLE IF EXISTS Article; DROP TABLE IF EXISTS Inproceedings; DROP TABLE IF EXISTS Authorship;
""")

# create tables
cursor.execute("""
CREATE TABLE Article (pubkey TEXT, title TEXT, journal TEXT, year INTEGER, PRIMARY KEY(pubkey));
CREATE TABLE Inproceedings (pubkey TEXT, title TEXT, booktitle TEXT, year INTEGER, PRIMARY KEY(pubkey));
CREATE TABLE Authorship (pubkey TEXT, author TEXT, PRIMARY KEY(pubkey,author));
""")

In [10]:
# create handler to process each article
class DBLPHandler (sax.ContentHandler):
    # important tags
    important_tags = ['title', 'booktitle', 'journal', 'year']
    
    # initialize values for the ones we need
    def __init__(self):
        # stores which type of tag we are in now
        self.currentData = ''
        
        # stores whether we are in article or inproceedings
        self.type = ''
        
        # storage of important tag values
        # individual will create a different row each time
        # initialize lists so it stores values as a row!
        self.pubkey = ''
        self.unique_authorship = [{'pubkey': 0, 'author': 1}]
        self.authorship = {'pubkey': '', 'author': ''}
        self.inproceedings = {'pubkey': '', 'title': '', 'booktitle': '', 'year': ''}
        self.article = {'pubkey': '', 'title': '', 'journal': '', 'year': ''}
        
    
    # read the element and figure out what tag it is
    def startElement(self, tag, attributes):
        self.currentData = tag
        if tag == 'article' or tag == 'inproceedings':
            # save tag for which content table to put it in later
            self.type = tag
            self.pubkey = attributes.get('key')
            
            # insert pubkey into each tuple
            self.authorship['pubkey'] = self.pubkey
            self.inproceedings['pubkey'] = self.pubkey
            self.article['pubkey'] = self.pubkey
    
    def endElement(self, tag):
        # ignore anything that's not article or inproceedings
        if self.type == 'article' or self.type == 'inproceedings':
            # multiple authors so need to treat those differently for a single article/inproceedings
            if tag == 'author':
                # create a list of unique pubkey-author pairs, to account for duplicates
                unique = True
                for dic in self.unique_authorship:
                    if self.authorship['pubkey'] == dic['pubkey'] and self.authorship['author'] == dic['author']:
                        unique = False
                        break
                
                if unique:
                    self.unique_authorship.append(self.authorship)
                
                # reset 
                self.authorship = {'pubkey': self.authorship['pubkey'], 'author': ''}
            
            # each of the other contents are unique for a pubkey
            # know that we're at the end of a full block (all authors taken), so we have extracted all content
            if tag == self.type:
                for dic in self.unique_authorship:
                    if dic['pubkey'] == 0 and dic['author'] == 1:
                        continue
                    
                    # set author to this pubkey (at the end of the block with list of authors)
                    vals = []
                    keys = '('
                    strings = '('
                    for key in dic.keys():
                        vals.append(dic[key])
                        if key == 'pubkey':
                            keys += key
                            strings += '%s'
                        else:
                            keys += ', ' + key
                            strings += ", %s"
                    strings += ')'
                    keys += ')'
                    vals = tuple(vals)
                    
                    cursor.execute('INSERT INTO Authorship ' + keys + ' VALUES ' + strings, vals)
                
                if self.type == 'inproceedings':
                    if self.inproceedings['year']:
                        self.inproceedings['year'] = int(self.inproceedings['year'])
                    
                    # set the values that are not null
                    vals = []
                    keys = '('
                    strings = '('
                    for key in self.inproceedings.keys():
                        if self.inproceedings[key]:
                            vals.append(self.inproceedings[key])
                            if key == 'pubkey':
                                keys += key
                                strings += '%s'
                            else:
                                keys += ', ' + key
                                strings += ", %s"
                    strings += ')'
                    keys += ')'
                    vals = tuple(vals)
                    
                    cursor.execute('INSERT INTO Inproceedings ' + keys + ' VALUES ' + strings, vals)
                else:
                    if self.article['year']:
                        self.article['year'] = int(self.article['year'])
                    
                    # set the values that are not null
                    vals = []
                    keys = '('
                    strings = '('
                    for key in self.article.keys():
                        if self.article[key]:
                            vals.append(self.article[key])
                            if key == 'pubkey':
                                keys += key
                                strings += '%s'
                            else:
                                keys += ', ' + key
                                strings += ", %s"
                    strings += ')'
                    keys += ')'
                    vals = tuple(vals)
                    
                    cursor.execute('INSERT INTO Article ' + keys + ' VALUES ' + strings, vals)

                # reset everything each block
                self.type = ''
                self.pubkey = ''
                self.unique_authorship = [{'pubkey': 0, 'author': 1}]
                self.authorship = {'pubkey': '', 'author': ''}
                self.inproceedings = {'pubkey': '', 'title': '', 'booktitle': '', 'year': ''}
                self.article = {'pubkey': '', 'title': '', 'journal': '', 'year': ''}
        
    # read content of each tag, to fill in the table
    def characters(self, content):
        content = content.replace('\n', '')
        if self.type == 'article' or self.type == 'inproceedings':
            if self.currentData == 'author':
                self.authorship['author'] += content
            elif self.type == 'inproceedings':
                # inproceedings we should have these three
                if self.currentData == 'title':
                    self.inproceedings['title'] += content
                elif self.currentData == 'booktitle':
                    self.inproceedings['booktitle'] += content
                elif self.currentData == 'year':
                    self.inproceedings['year'] += content
            else:
                # article we should have these three
                if self.currentData == 'title':
                    self.article['title'] += content
                elif self.currentData == 'journal':
                    self.article['journal'] += content
                elif self.currentData == 'year':
                    self.article['year'] += content

In [11]:
# create parser
reader = sax.make_parser()

# create handler and set
reader.setContentHandler(DBLPHandler())

# parse
reader.parse('../../Files/dblp-2022-01-01.xml')

In [12]:
# commit changes
connection.commit()

# close the database
cursor.close()
connection.close()

In [13]:
# connect to database
connection = psycopg2.connect("dbname = dblp user = dblpuser password = dblp_password")

# cursor to perform operations in postgres
cursor = connection.cursor()

# count right for answer checking in Python so I dob't have to wait for pgAdmin to load each time
# query count and name the attribute "cnt" instead of the standard "count"
cursor.execute("""
SELECT COUNT(*) as cnt from "Inproceedings";
""")
inproceedings_row_count = "inproceedings row " + str(cursor.description[0][0]) + ": " + str(cursor.fetchone()[0])

cursor.execute("""
SELECT COUNT(*) as cnt from "Article";
""")
article_row_count = "article row " + str(cursor.description[0][0]) + ": " + str(cursor.fetchone()[0])

cursor.execute("""
SELECT COUNT(*) as cnt from "Authorship";
""")
authorship_row_count =  "authorship row " + str(cursor.description[0][0]) + ": " + str(cursor.fetchone()[0])

print(inproceedings_row_count)
print(article_row_count)
print(authorship_row_count)

# close the database
cursor.close()
connection.close()

inproceedings row cnt: 2956396
article row cnt: 2738932
authorship row cnt: 18128940
